## Import

In [3]:
!pip install yfinance
!pip install finance-datareader
!pip install pykrx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 11.5 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import random
import os

import pickle

from tqdm import tqdm
from statsmodels.tsa.arima.model import ARIMA

#증권시장의 주식정보 스크래핑
#pip install pykrx
from pykrx import stock
from pykrx import bond

import yfinance as yf
import FinanceDataReader as fdr
import pykrx

import warnings
warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

## Data Load

In [48]:
# train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')

In [49]:
# train.head(20)

,일자,종목코드,종목명,거래량,시가,고가,저가,종가
0,20210601,A060310,3S,166690,2890,2970,2885,2920
1,20210601,A095570,AJ네트웍스,63836,5860,5940,5750,5780
2,20210601,A006840,AK홀딩스,103691,35500,35600,34150,34400
3,20210601,A054620,APS,462544,14600,14950,13800,14950
4,20210601,A265520,AP시스템,131987,29150,29150,28800,29050
5,20210601,A211270,AP위성,623179,15300,16100,14950,16100
6,20210601,A027410,BGF,345706,7020,7150,7000,7050
7,20210601,A282330,BGF리테일,39293,182500,188500,181500,184000
8,20210601,A126600,BGF에코머티리얼즈,632510,6410,6540,6280,6400
9,20210601,A138930,BNK금융지주,1299625,7940,8000,7890,7990


In [ ]:

# # 각 종목별 OHLCV 데이터를 가져오기
# def get_ohlcv(tickers, start_date, end_date):
#     price_dataframes = []

#     for ticker in tickers:
#         print(f'Processing {ticker}')
#         price_df = stock.get_market_ohlcv_by_date(start_date, end_date, ticker)
#         price_df.reset_index(inplace=True)
#         price_df['종목코드'] = ticker

#         price_dataframes.append(price_df)
#     combined_price_df = pd.concat(price_dataframes, ignore_index=True)

#     return combined_price_df

# start_date = '20210601'
# end_date = '20230530'
# train_tickers = train['Ticker'].unique().tolist()  # 중복 제거하여 티커 값 리스트로 변환
# train = get_ohlcv(train_tickers, start_date, end_date)


In [ ]:
# train.head(10)

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률,종목코드
0,2021-06-01,2890,2970,2885,2920,166690,487455970,1.04,060310
1,2021-06-02,2915,2975,2830,2900,134833,388795625,-0.68,060310
2,2021-06-03,2900,2925,2875,2900,144470,419668300,0.00,060310
3,2021-06-04,2930,3120,2920,2950,934224,2840373820,1.72,060310
4,2021-06-07,3000,3150,2955,3150,946560,2929678225,6.78,060310
5,2021-06-08,3155,3180,3035,3115,422752,1322783350,-1.11,060310
6,2021-06-09,3105,3280,3080,3105,655673,2097959840,-0.32,060310
7,2021-06-10,3105,3185,3010,3150,314027,975190275,1.45,060310
8,2021-06-11,3145,3175,3060,3085,302485,936045120,-2.06,060310
9,2021-06-14,3085,3100,3030,3080,174962,535956850,-0.16,060310


In [ ]:
# import pickle
# # 데이터프레임을 pickle 파일로 저장 (이유: 각 종목별 OHLCV 데이터를 가져오기가 오래걸려서 (45분정도 소요))
# pickle.dump(train, open('/content/drive/MyDrive/Colab Notebooks/train_data.pkl', 'wb'))

In [50]:
# # 저장된 pickle 파일 불러오기
# loaded_train = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/train_data.pkl', 'rb'))

In [51]:
# train = loaded_train
# train

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률,종목코드
0,2021-06-01,2890,2970,2885,2920,166690,487455970,1.040000,060310
1,2021-06-02,2915,2975,2830,2900,134833,388795625,-0.680000,060310
2,2021-06-03,2900,2925,2875,2900,144470,419668300,0.000000,060310
3,2021-06-04,2930,3120,2920,2950,934224,2840373820,1.720000,060310
4,2021-06-07,3000,3150,2955,3150,946560,2929678225,6.780000,060310
...,...,...,...,...,...,...,...,...,...
987995,2023-05-23,6710,6710,6420,6430,40981,266207710,-4.170000,238490
987996,2023-05-24,6400,6490,6210,6320,19392,123635550,-1.710000,238490
987997,2023-05-25,6340,6400,6270,6330,6457,40862130,0.160000,238490
987998,2023-05-26,6330,6410,6300,6330,8905,56621540,0.000000,238490


In [52]:
# def get_mv(df):
#     """
#     이동평균을 계산하여 데이터프레임에 추가하는 함수

#     Parameters:
#         - df: 이동평균을 계산할 데이터프레임
#     Returns:
#         계산된 이동평균 값을 포함한 데이터프레임
#     """
#     grouped = df.groupby('종목코드')  # 종목별로 그룹화
#     df['Close_mv5'] = grouped['종가'].rolling(5, min_periods=5).mean().reset_index(0, drop=True) # 5일 이동평균 계산
#     df['Close_mv10'] = grouped['종가'].rolling(10, min_periods=10).mean().reset_index(0, drop=True)  # 10일 이동평균 계산
#     df['Close_mv20'] = grouped['종가'].rolling(20, min_periods=20).mean().reset_index(0, drop=True)  # 20일 이동평균 계산

#     return df


# train = get_mv(train)
# train

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률,종목코드,Close_mv5,Close_mv10,Close_mv20
0,2021-06-01,2890,2970,2885,2920,166690,487455970,1.040000,060310,NaN,NaN,NaN
1,2021-06-02,2915,2975,2830,2900,134833,388795625,-0.680000,060310,NaN,NaN,NaN
2,2021-06-03,2900,2925,2875,2900,144470,419668300,0.000000,060310,NaN,NaN,NaN
3,2021-06-04,2930,3120,2920,2950,934224,2840373820,1.720000,060310,NaN,NaN,NaN
4,2021-06-07,3000,3150,2955,3150,946560,2929678225,6.780000,060310,2964.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
987995,2023-05-23,6710,6710,6420,6430,40981,266207710,-4.170000,238490,6540.0,6416.0,6440.5
987996,2023-05-24,6400,6490,6210,6320,19392,123635550,-1.710000,238490,6548.0,6399.0,6429.5
987997,2023-05-25,6340,6400,6270,6330,6457,40862130,0.160000,238490,6502.0,6383.0,6421.0
987998,2023-05-26,6330,6410,6300,6330,8905,56621540,0.000000,238490,6424.0,6393.0,6419.5


In [53]:
# def get_obv(train):#OBV 컬럼에 추가하기
#     #종목코드 2000가지를 가져오자
#     stock_code = train['종목코드'].unique()

#     #데이터 프레임에 틀 만들기
#     train['OBV'] = 0
#     train['OBV_EMA'] = 0

#     for code in tqdm(stock_code): #각 종목별, 총 2000회 반복
#         df_train = train[train['종목코드'] == code]
#         OBV = []
#         OBV.append(0) #[-1]을 사용하기 위해 초기 값을 넣어준다.
#         for i in range(1, len(df_train['종가'])): #기간 만큼 돌려준다.
#             if df_train['종가'].iloc[1] > df_train['종가'].iloc[i-1]: #현재 종가가 전일 종가보다 크다면
#                 OBV.append(OBV[-1] + df_train['거래량'].iloc[i]) #OBV 제일 마지막 값에 현재일 거래량 값을 더한 값을 OBV에 추가.
#             elif df_train['종가'].iloc[1] < df_train['종가'].iloc[i-1]: #현재 종가가 전일 종가보다 작다면
#                 OBV.append(OBV[-1] - df_train['거래량'].iloc[i]) #OBV 제일 마지막 값에 현재일 거래량 값을 빼준 값을 OBV에 추가.
#             else: #종가가 동일하다면
#                 OBV.append(OBV[-1]) #OBV 제일 마지막 값을 다시 한번 OBV에 추가.

#         train.loc[train['종목코드'] == code, 'OBV'] = OBV #OBV 리스트 안의 값을 코드에 넣어주자.
#         train.loc[train['종목코드'] == code,'OBV_EMA'] = train[train['종목코드'] == code]['OBV'].ewm(com=20).mean() #OBV의 이동평균 값을 넣어주자.
#         return train


# # forex 데이터
# def get_forex_index(df):
#     start_date_yf = '2021-06-01'
#     end_date_yf = '2023-05-31'

#     start_date = '20210601'
#     end_date = '20230531'

#     forex_index_data = yf.download(["USDKRW=X", "USDAUD=X", "USDJPY=X", "EURUSD=X", "CNY=X", "^GSPC", "^DJI", "^IXIC", "^STOXX50E",
#                                    "^SOX",  "000001.SS", "000300.SS", "MME=F", "^TNX"],
#                                   start=start_date_yf, end=end_date_yf, rounding=True)

#     tmp_forex_index = forex_index_data["Close"]
#     tmp_forex_index.index = pd.to_datetime(tmp_forex_index.index)
#     tmp_forex_index = tmp_forex_index[(tmp_forex_index.index >= pd.to_datetime(start_date)) &
#                                       (tmp_forex_index.index <= pd.to_datetime(end_date))]
#     tmp_forex_index.columns = ["sse_composite_index", "csi300_index", "usdtocny", "eurtousd", "msci_emerging",
#                                "usdtoaud", "usdtojpy", "usdtokrw",
#                                "dow", "snp500", "nasdaq", "semicon_index", "euro50", "us10y_tsy"]

#     tmp_forex_index.reset_index(drop=False, inplace=True)

#     merged_df = df.merge(tmp_forex_index, left_on='날짜', right_on='Date', how='left')
#     merged_df.drop(columns=['Date'], inplace=True)
#     return merged_df



# #스토캐스틱 fast 지표
# def sto_fast_logic(df_fast):
#     n = 14
#     m = 5

#     #n일중 최고가
#     ndays_high = df_fast['고가'].rolling(window = n, min_periods = 1).max()
#     #n일중 최저가
#     ndays_low = df_fast['저가'].rolling(window = n, min_periods = 1).min()
#     #Fast%K 계산
#     fast_k = ((df_fast['종가'] - ndays_low) / (ndays_high - ndays_low)) * 100
#     #Fast%D 계산
#     fast_d = fast_k.ewm(span = m).mean()

#     return (fast_k, fast_d)

# def get_sto_fast(df):
#     df['fast_k'] = 0
#     df['fast_d'] = 0
#     stock_code = df['종목코드'].unique()

#     for code in tqdm(stock_code): #각 종목별, 총 2000회 반복
#         df_fast = df[df['종목코드'] == code]
#         (fast_k, fast_d) = sto_fast_logic(df_fast)
#         df.loc[df['종목코드'] == code, 'fast_k'] = fast_k
#         df.loc[df['종목코드'] == code,'fast_d'] = fast_d
#     return df

# #스토캐스틱 slow 지표
# def sto_slow_logic(df_slow):
#     n = 14
#     m = 5
#     t = 5

#     #n일중 최고가
#     ndays_high = df_slow['고가'].rolling(window = n, min_periods = 1).max()
#     #n일중 최저가
#     ndays_low = df_slow['저가'].rolling(window = n, min_periods = 1).min()
#     #Fast%K 계산
#     fast_k = ((df_slow['종가'] - ndays_low) / (ndays_high - ndays_low)) * 100
#     #Slow%K 계산
#     slow_k = fast_k.ewm(span = m).mean()
#     #Slow%D 계산
#     slow_d = slow_k.ewm(span = t).mean()

#     return (slow_k, slow_d)

# def get_sto_slow(df):
#     df['slow_k'] = 0
#     df['slow_d'] = 0
#     stock_code = df['종목코드'].unique()

#     for code in tqdm(stock_code): #각 종목별, 총 2000회 반복
#         df_slow = df[df['종목코드'] == code]
#         (slow_k, slow_d) = sto_fast_logic(df_slow)
#         df.loc[df['종목코드'] == code, 'slow_k'] = slow_k
#         df.loc[df['종목코드'] == code,'slow_d'] = slow_d
#     return df


# train = get_obv(train)
# train = get_forex_index(train)
# train = get_sto_fast(train)


  0%|          | 0/2000 [00:00<?, ?it/s]


[*********************100%***********************]  14 of 14 completed


100%|██████████| 2000/2000 [09:06<00:00,  3.66it/s]


In [55]:
# train.isnull().sum()

날짜                         0
시가                         0
고가                         0
저가                         0
종가                         0
거래량                        0
거래대금                       0
등락률                        0
종목코드                       0
Close_mv5               8000
Close_mv10             18000
Close_mv20             38000
OBV                        0
OBV_EMA                    0
sse_composite_index    50000
csi300_index           50000
usdtocny                   0
eurtousd                   0
msci_emerging          22000
usdtoaud                   0
usdtojpy                   0
usdtokrw                   0
dow                    34000
snp500                 34000
nasdaq                 34000
semicon_index          34000
euro50                 20000
us10y_tsy              34000
fast_k                     1
fast_d                 14402
dtype: int64

In [56]:
# pickle.dump(train, open('/content/drive/MyDrive/Colab Notebooks/train_data_df2.pkl', 'wb'))

In [6]:
loaded_train_df = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/train_data_df2.pkl', 'rb'))

In [7]:
loaded_train_df

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률,종목코드,Close_mv5,...,usdtojpy,usdtokrw,dow,snp500,nasdaq,semicon_index,euro50,us10y_tsy,fast_k,fast_d
0,2021-06-01,2890,2970,2885,2920,166690,487455970,1.040000,060310,NaN,...,109.493,1106.9800,34575.31,4202.04,13736.48,3174.32,4071.75,1.615,41.176471,41.176471
1,2021-06-02,2915,2975,2830,2900,134833,388795625,-0.680000,060310,NaN,...,109.493,1107.2100,34600.38,4208.12,13756.33,3196.98,4088.50,1.591,48.275862,45.436105
2,2021-06-03,2900,2925,2875,2900,144470,419668300,0.000000,060310,NaN,...,109.595,1109.8800,34577.04,4192.85,13614.51,3139.19,4079.24,1.625,48.275862,46.781253
3,2021-06-04,2930,3120,2920,2950,934224,2840373820,1.720000,060310,NaN,...,110.266,1115.4399,34756.39,4229.89,13814.49,3214.14,4089.38,1.560,41.379310,44.537369
4,2021-06-07,3000,3150,2955,3150,946560,2929678225,6.780000,060310,2964.0,...,109.617,1109.6801,34630.24,4226.52,13881.72,3191.66,4097.65,1.569,100.000000,65.828711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987995,2023-05-23,6710,6710,6420,6430,40981,266207710,-4.170000,238490,6540.0,...,138.672,1313.8300,33055.51,4145.58,12560.25,3177.44,4342.38,3.698,55.789474,64.061314
987996,2023-05-24,6400,6490,6210,6320,19392,123635550,-1.710000,238490,6548.0,...,138.598,1319.8400,32799.92,4115.24,12484.16,3124.09,4263.74,3.719,44.210526,57.444385
987997,2023-05-25,6340,6400,6270,6330,6457,40862130,0.160000,238490,6502.0,...,139.283,1318.5200,32764.65,4151.28,12698.09,3336.72,4269.64,3.814,45.263158,53.383976
987998,2023-05-26,6330,6410,6300,6330,8905,56621540,0.000000,238490,6424.0,...,140.017,1330.8900,33093.34,4205.45,12975.69,3545.67,4337.50,3.810,45.263158,50.677037


In [8]:
train = loaded_train_df

In [12]:
print(train.columns)


Index(['날짜', '시가', '고가', '저가', '종가', '거래량', '거래대금', '등락률', '종목코드', 'Close_mv5',
       'Close_mv10', 'Close_mv20', 'OBV', 'OBV_EMA', 'sse_composite_index',
       'csi300_index', 'usdtocny', 'eurtousd', 'msci_emerging', 'usdtoaud',
       'usdtojpy', 'usdtokrw', 'dow', 'snp500', 'nasdaq', 'semicon_index',
       'euro50', 'us10y_tsy', 'fast_k', 'fast_d'],
      dtype='object')


In [13]:
train

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률,종목코드,Close_mv5,...,usdtojpy,usdtokrw,dow,snp500,nasdaq,semicon_index,euro50,us10y_tsy,fast_k,fast_d
0,2021-06-01,2890,2970,2885,2920,166690,487455970,1.040000,060310,NaN,...,109.493,1106.9800,34575.31,4202.04,13736.48,3174.32,4071.75,1.615,41.176471,41.176471
1,2021-06-02,2915,2975,2830,2900,134833,388795625,-0.680000,060310,NaN,...,109.493,1107.2100,34600.38,4208.12,13756.33,3196.98,4088.50,1.591,48.275862,45.436105
2,2021-06-03,2900,2925,2875,2900,144470,419668300,0.000000,060310,NaN,...,109.595,1109.8800,34577.04,4192.85,13614.51,3139.19,4079.24,1.625,48.275862,46.781253
3,2021-06-04,2930,3120,2920,2950,934224,2840373820,1.720000,060310,NaN,...,110.266,1115.4399,34756.39,4229.89,13814.49,3214.14,4089.38,1.560,41.379310,44.537369
4,2021-06-07,3000,3150,2955,3150,946560,2929678225,6.780000,060310,2964.0,...,109.617,1109.6801,34630.24,4226.52,13881.72,3191.66,4097.65,1.569,100.000000,65.828711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987995,2023-05-23,6710,6710,6420,6430,40981,266207710,-4.170000,238490,6540.0,...,138.672,1313.8300,33055.51,4145.58,12560.25,3177.44,4342.38,3.698,55.789474,64.061314
987996,2023-05-24,6400,6490,6210,6320,19392,123635550,-1.710000,238490,6548.0,...,138.598,1319.8400,32799.92,4115.24,12484.16,3124.09,4263.74,3.719,44.210526,57.444385
987997,2023-05-25,6340,6400,6270,6330,6457,40862130,0.160000,238490,6502.0,...,139.283,1318.5200,32764.65,4151.28,12698.09,3336.72,4269.64,3.814,45.263158,53.383976
987998,2023-05-26,6330,6410,6300,6330,8905,56621540,0.000000,238490,6424.0,...,140.017,1330.8900,33093.34,4205.45,12975.69,3545.67,4337.50,3.810,45.263158,50.677037


In [9]:
from pykrx import stock

def add_market_cap(df):
    df['시가총액'] = 0

    stock_code = df['종목코드'].unique()

    for code in tqdm(stock_code): #각 종목별, 총 2000회 반복
        market_cap = stock.get_market_cap('20210601', '20230530', str(code))
        market_cap_only = list(market_cap['시가총액'])

        df.loc[df['종목코드'] == code, '시가총액'] = market_cap_only
    return df

train = add_market_cap(train)

100%|██████████| 2000/2000 [42:20<00:00,  1.27s/it]


In [10]:
train

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률,종목코드,Close_mv5,...,usdtokrw,dow,snp500,nasdaq,semicon_index,euro50,us10y_tsy,fast_k,fast_d,시가총액
0,2021-06-01,2890,2970,2885,2920,166690,487455970,1.040000,060310,NaN,...,1106.9800,34575.31,4202.04,13736.48,3174.32,4071.75,1.615,41.176471,41.176471,135112817960
1,2021-06-02,2915,2975,2830,2900,134833,388795625,-0.680000,060310,NaN,...,1107.2100,34600.38,4208.12,13756.33,3196.98,4088.50,1.591,48.275862,45.436105,134187387700
2,2021-06-03,2900,2925,2875,2900,144470,419668300,0.000000,060310,NaN,...,1109.8800,34577.04,4192.85,13614.51,3139.19,4079.24,1.625,48.275862,46.781253,134187387700
3,2021-06-04,2930,3120,2920,2950,934224,2840373820,1.720000,060310,NaN,...,1115.4399,34756.39,4229.89,13814.49,3214.14,4089.38,1.560,41.379310,44.537369,136500963350
4,2021-06-07,3000,3150,2955,3150,946560,2929678225,6.780000,060310,2964.0,...,1109.6801,34630.24,4226.52,13881.72,3191.66,4097.65,1.569,100.000000,65.828711,145755265950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987995,2023-05-23,6710,6710,6420,6430,40981,266207710,-4.170000,238490,6540.0,...,1313.8300,33055.51,4145.58,12560.25,3177.44,4342.38,3.698,55.789474,64.061314,72737677480
987996,2023-05-24,6400,6490,6210,6320,19392,123635550,-1.710000,238490,6548.0,...,1319.8400,32799.92,4115.24,12484.16,3124.09,4263.74,3.719,44.210526,57.444385,71493331520
987997,2023-05-25,6340,6400,6270,6330,6457,40862130,0.160000,238490,6502.0,...,1318.5200,32764.65,4151.28,12698.09,3336.72,4269.64,3.814,45.263158,53.383976,71606453880
987998,2023-05-26,6330,6410,6300,6330,8905,56621540,0.000000,238490,6424.0,...,1330.8900,33093.34,4205.45,12975.69,3545.67,4337.50,3.810,45.263158,50.677037,71606453880


In [11]:
pickle.dump(train, open('/content/drive/MyDrive/Colab Notebooks/train_data_df2_시가총액추가.pkl', 'wb'))